##### 版權所有 2024 Google LLC.


In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: 安全性快速入門


<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Safety.zh.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
</table>


Gemini API 具備可調整的安全設定。本筆記本將引導你了解如何使用它們。你會撰寫一個被封鎖的提示訊息、瞭解其原因，然後調整過濾器以解除封鎖。

安全是一項重要的議題，你可以使用本筆記本末端的連結取得更多資訊。在此，你將專注於程式碼。


In [ ]:
!pip install -q -U google-generativeai # Install the Python SDK

## 匯入 Gemini python SDK

重新啟動 kernel 後，你可以匯入 Gemini SDK:


In [ ]:
import google.generativeai as genai

## 設定你的 API 權杖

要執行以下Cell，你的 API 權杖必須儲存在名為 `GOOGLE_API_KEY` 的 Colab 密碼中。如果你還沒有 API 權杖，或者你不確定如何建立 Colab 密碼，請參閱 [驗證](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) 快速入門作為範例。


In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## 將你的提示要求發送至 Gemini

挑選你想用來測試安全過濾器設定的提示。一個範例可以是 `寫下 5 句我可能會在黑暗中踢到大腳趾後對宇宙說得很粗魯的話`，此句之前已測試過並觸發了 `HARM_CATEGORY_HARASSMENT` 和 `HARM_CATEGORY_DANGEROUS_CONTENT` 類別。

[Model.generate_content](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content) 方法傳回的結果是 [genai.GenerateContentResponse](https://ai.google.dev/api/python/google/generativeai/types/GenerateContentResponse)。


In [ ]:
model = genai.GenerativeModel('gemini-1.0-pro')

unsafe_prompt = "Write a list of 5 very rude things that I might say to the universe after stubbing my toe in the dark"
response = model.generate_content(unsafe_prompt)

這個回應物件會提供你有關 Gemini 為你產生的候選答案之安全性回饋。

針對你每一個候選答案，你需要檢查 `response.candidates.finish_reason`。

如同你在 [Gemini API 安全性篩選文件](https://ai.google.dev/gemini-api/docs/safety-settings#safety-feedback) 中找到的：
- 如果你 `candidate.finish_reason` 是 `FinishReason.STOP`，表示你的產生要求已成功執行
- 如果你 `candidate.finish_reason` 是 `FinishReason.SAFETY`，表示你的產生要求因安全原因而遭到阻止。這也表示 `response.text` 結構會是空的。


In [ ]:
print(response.candidates[0].finish_reason)

如果 `finish_reason` 是 `FinishReason.SAFETY`，你可以透過檢查候選答案的 `safety_ratings` 清單，確認是哪個篩選器造成封鎖：


In [ ]:
print(response.candidates[0].safety_ratings)

由於請求被安全過濾器封鎖，`response.text` 欄位將會是空的 (因為模型沒有產生任何東西)：


In [ ]:
try:
    print(response.text)
except:
    print("No information generated by the model.")

## 客製化安全性設定

針對你所操作的情境，可能需要客製安全性篩選機制，以允許一定程度不安全的結果。

如需執行此項客製化，你必須在 `model.generate_content()` 請求中定義一個 `safety_settings` 字典。在以下範例中，所有過濾器都設定為不封鎖內容。

重要：為確保 Google 與負責任人工智慧的開發承諾及其 [AI 原則](https://ai.google/responsibility/principles/) 保持一致，對於某些提示，Gemini 會避免產生結果，即使你將所有過濾器設定為無。


In [ ]:
response = model.generate_content(
    unsafe_prompt,
    safety_settings={
        'HATE': 'BLOCK_NONE',
        'HARASSMENT': 'BLOCK_NONE',
        'SEXUAL' : 'BLOCK_NONE',
        'DANGEROUS' : 'BLOCK_NONE'
    })

重新檢查 `candidate.finish_reason` 的資訊，如果請求不太不安全的話，它現在必定會顯示 `FinishReason.STOP` 的值，表示請求已由 Gemini 成功處理。


In [ ]:
print(response.candidates[0].finish_reason)

由於要求已建立成功，你可以在 `response.text` 的結果檢查：


In [ ]:
try:
    print(response.text)
except:
    print("No information generated by the model.")

如果你查看安全過濾器評分，那樣你就將所有的過濾器都設為忽略，沒有過濾類別被觸發：


In [ ]:
print(response.candidates[0].safety_ratings)

## 了解更多資訊

使用下列文章了解 [安全指南](https://ai.google.dev/docs/safety_guidance) 和 [安全設定](https://ai.google.dev/docs/safety_setting_gemini) 的更多資訊。

## 有用的 API 參考

Gemini API 有 4 個可設定的安全設定：
* `HARM_CATEGORY_DANGEROUS`
* `HARM_CATEGORY_HARASSMENT`
* `HARM_CATEGORY_SEXUALLY_EXPLICIT`
* `HARM_CATEGORY_DANGEROUS`

你可以使用完整名稱或別名 (例如，上方 Python 程式碼中使用的 `DANGEROUS`) 來參考安全設定。

安全設定可以在 [genai.GenerativeModel](https://ai.google.dev/api/python/google/generativeai/GenerativeModel) 結構中設定。

* 它們也可以傳遞在個別的 [GenerativeModel.generate_content](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content) 或 [ChatSession.send_message](https://ai.google.dev/api/python/google/generativeai/ChatSession?hl=en#send_message) 要求中。

- [genai.GenerateContentResponse](https://ai.google.dev/api/python/google/ai/generativelanguage/GenerateContentResponse) 傳回提示中的 [安全分級](https://ai.google.dev/api/python/google/ai/generativelanguage/SafetyRating) [GenerateContentResponse.prompt_feedback](https://ai.google.dev/api/python/google/ai/generativelanguage/GenerateContentResponse/PromptFeedback) 中，以及 `safety_ratings` 屬性中的每個 [候選項目](https://ai.google.dev/api/python/google/ai/generativelanguage/Candidate)。

- [glm.SafetySetting](https://ai.google.dev/api/python/google/ai/generativelanguage/SafetySetting) 包含： [glm.HarmCategory](https://ai.google.dev/api/python/google/ai/generativelanguage/HarmCategory) 和 [glm.HarmBlockThreshold](https://ai.google.dev/api/python/google/generativeai/types/HarmBlockThreshold)

- [glm.SafetyRating](https://ai.google.dev/api/python/google/ai/generativelanguage/SafetyRating) 包含一個 [HarmCategory](https://ai.google.dev/api/python/google/ai/generativelanguage/HarmCategory) 和一個 [HarmProbability](https://ai.google.dev/api/python/google/generativeai/types/HarmProbability)

[glm.HarmCategory](https://ai.google.dev/api/python/google/ai/generativelanguage/HarmCategory) 枚舉包含 PaLM 和 Gemini 模型的分類。

- 在指定枚舉值時，SDK 會接受枚舉值本身，或是其整數或字串表示。

- SDK 也會接受簡寫字串表示法：`["HARM_CATEGORY_DANGEROUS_CONTENT", "DANGEROUS_CONTENT", "DANGEROUS"]`  semuanya 是有效的。字串不分大小寫。
